# Adding Messaging capabilities to AG2 agents

In this notebook we introduce send and retrieve message tools for 3 messaging platforms: Discord, Slack, and Telegram.

These tools can be added to your AG2 agents, enabling them to send messages during a workflow and/or retrieve messages for use by the workflow.

There's a two types of tools with one for each platform:
- Sending a message (`DiscordSendTool`, `SlackSendTool`, `TelegramSendTool`)
- Retrieving messages (`DiscordRetrieveTool`, `SlackRetrieveTool`, `TelegramRetrieveTool`)

You can attach one or more of them to an agent and with multiple tool-calling capable LLMs, though it may be most typical that you create an agent for each platform and attach the send and/or retrieve tool. This allows you to tailor your system message for your agent to the messaging style used for the platform.

## Installation

Each messaging platform requires installation of a package specifically for that platform.

To use the Discord tools install AG2 with the `commsagent-discord` extra:
```bash
pip install ag2[commsagent-discord]
```

Similarly, for slack and/or telegram:
```bash
pip install ag2[commsagent-slack]
```
and
```bash
pip install ag2[commsagent-telegram]
```

You're all set! Now you can start sending to and retrieving messages from these platforms.

## Discord Send/Retrieve Messages

### Tokens, Guilds, and Channels
The Discord tools, `DiscordSendTool` and `DiscordRetrieveTool`, require authentication (bot token), server (guild name), and channel details in order to send/retrieve messages.

Here are some references to help you establish those details:
- [Creating a Bot Account](https://discordpy.readthedocs.io/en/latest/discord.html), and [another reference](https://gist.github.com/SinisterRectus/2d84025f51bc0be37d2f0d0c559c2e35#how-do-i-make-a-discord-bot)
- [Adding a Bot to a server](https://gist.github.com/SinisterRectus/2d84025f51bc0be37d2f0d0c559c2e35#how-do-i-add-a-discord-bot-to-my-server)
- Bot Permissions you will need:
  - General Permissions: View Channels
  - Text Permissions: Send Messages, Read Message History, Use Slash Commands
- Server/Guild Name - Right-click the Server > Server Settings > Overview, see the Server Name at the top right (e.g. "My Test Server")
- Channel name - The text after the hash (e.g. "general")

### Sending a message to Discord

Let's add the Discord Send and Retrieve tools to an agent and have them executed by another agent in as simple 2-agent chat.

In [ ]:
import os

from autogen import AssistantAgent, UserProxyAgent
from autogen.tools.experimental import BrowserUseTool

## Imports

In [ ]:
import os

from autogen import AssistantAgent, UserProxyAgent
from autogen.tools.experimental import BrowserUseTool

### Agent Configuration

Configure the agents for the interaction.

- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "config_list": config_list,
}

user_proxy = UserProxyAgent(name="user_proxy", human_input_mode="NEVER")
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

## Integrating Web Browsing with BrowserUseTool


The `BrowserUseTool` enables agents to interact with web browsers, allowing them to access, navigate, and perform actions on websites as part of their tasks. It acts as a bridge between the language model and the browser, empowering the agent to browse the web, search for information, and interact with dynamic web content.

To see what the agents are doing in real-time, set the `headless` option within the `browser_config` to `False`. This ensures that the browser runs in a visible window, allowing you to observe the agents' interactions with the websites. By default, setting `headless=True` would run the browser in the background without a GUI, useful for automated tasks where visibility is not necessary.

In [ ]:
browser_use_tool = BrowserUseTool(
    llm_config=llm_config,
    browser_config={"headless": False},
)

browser_use_tool.register_for_execution(user_proxy)
browser_use_tool.register_for_llm(assistant)

## Initiate Chat

For running the code in Jupyther, use `nest_asyncio` to allow nested event loops.

```bash
pip install nest_asyncio
```

In [4]:
import nest_asyncio

nest_asyncio.apply()

The `user_proxy.initiate_chat()` method triggers the assistant to perform a web browsing task, such as searching for "AG2" on Reddit, clicking the first post, and extracting the first comment. The assistant then executes the task using the `BrowserUseTool` and returns the extracted content to the user.

In [ ]:
result = user_proxy.initiate_chat(
    recipient=assistant,
    message="Go to Reddit, search for 'ag2' in the search bar, click on the first post and return the first comment.",
    max_turns=2,
)